In [1]:
import os

In [2]:
%pwd

'/Users/sushanth/Desktop/MLOps/Yoga_Pose_Image_Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sushanth/Desktop/MLOps/Yoga_Pose_Image_Classification'

In [17]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sushanth-0/Yoga_Pose_Image_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sushanth-0"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1df6ac3360167070b263f4f63b1a6628069a2946"

In [18]:
import tensorflow as tf

In [19]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: tuple
    params_batch_size: int

In [21]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [22]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Yoga dataset",
            mlflow_uri="https://dagshub.com/sushanth-0/Yoga_Pose_Image_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [23]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [43]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [46]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-13 17:02:16,277: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-13 17:02:16,279: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-13 17:02:16,279: INFO: common: created directory at: artifacts]
Found 1749 images belonging to 107 classes.
55/55 [==============================] - 13s 225ms/step - loss: 1.5127 - accuracy: 0.6249
[2024-06-13 17:02:29,944: INFO: common: json file saved at: scores.json]


2024/06/13 17:02:30 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/58/fc1l05pd7zb81y2wfqc56s480000gn/T/tmp0ok66u4s/model/data/model/assets
[2024-06-13 17:02:31,169: INFO: builder_impl: Assets written to: /var/folders/58/fc1l05pd7zb81y2wfqc56s480000gn/T/tmp0ok66u4s/model/data/model/assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [49]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [50]:
import tensorflow as tf
import mlflow

print("TensorFlow version:", tf.__version__)
print("MLflow version:", mlflow.__version__)


TensorFlow version: 2.13.0
MLflow version: 2.2.2


In [56]:
import tensorflow as tf
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse
from pathlib import Path
import yaml
import json
from dataclasses import dataclass
from pathlib import Path

def read_yaml(filepath):
    with open(filepath, 'r') as file:
        return yaml.safe_load(file)

def create_directories(paths):
    for path in paths:
        Path(path).mkdir(parents=True, exist_ok=True)

def save_json(path, data):
    with open(path, 'w') as file:
        json.dump(data, file)

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: tuple
    params_batch_size: int
    
    
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Yoga dataset",
            mlflow_uri="https://dagshub.com/sushanth-0/Yoga_Pose_Image_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: str) -> tf.keras.Model:
        path = str(path)  # Ensure path is a string
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Log the model with a registered model name
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                # Log the model without registering it
                mlflow.tensorflow.log_model(self.model, "model")

# Example usage in the script
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

AttributeError: 'dict' object has no attribute 'artifacts_root'